In [1]:
%pip install -q kagglehub
print('done')

done


In [11]:
import amazon_dataset
X, y = amazon_dataset.load_amazon_reviews()

X = X.sample(n = 20000, random_state=1)
y = y.sample(n = 20000, random_state=1)
print(X)
print(y)

Dataset downloaded to: /kaggle/input/amazon-fine-food-reviews
Loading data from /kaggle/input/amazon-fine-food-reviews/Reviews.csv...
Original dataset shape: (568454, 10)
Shape after filtering out neutral reviews (Score=3): (525814, 10)
Data processing complete. Returning text reviews and binary sentiment labels.
87500     This is one of my favorite flavors.  This Fren...
476116    I bought this for my girl friend that recently...
225031    Not a big fan of chili actually... so these al...
155722    The tea helped me tremendously with getting a ...
55081     I use curry powder in a lot of my cooking - fr...
                                ...                        
227313    My son is on a very restricted diet, and this ...
400922    I like cats.  I like dogs.  I just don't like ...
45510     I was not sure what to expect with this drink ...
208236    I was surprised that these were flavored so po...
91683     The title says it all!!<br /><br />If you pay ...
Name: Text, Length: 20000

In [13]:
import pandas as pd
import numpy as np
import re
import string

#import dataset
#data = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["comments", "student_star"])


#preprocessing
#remove all punctuation
X = X.str.replace(r'[^\w\s]+', '', regex = True)
#make all letters lowercase
X = X.str.lower()

print(X)

87500     this is one of my favorite flavors  this frenc...
476116    i bought this for my girl friend that recently...
225031    not a big fan of chili actually so these almon...
155722    the tea helped me tremendously with getting a ...
55081     i use curry powder in a lot of my cooking  fro...
                                ...                        
227313    my son is on a very restricted diet and this c...
400922    i like cats  i like dogs  i just dont like my ...
45510     i was not sure what to expect with this drink ...
208236    i was surprised that these were flavored so po...
91683     the title says it allbr br if you pay less you...
Name: Text, Length: 20000, dtype: object


In [14]:
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#list of stop words
stop_str = (['the', 'a', 'and', 'to', 'you', 'is', 'he', 'she', 'of', 'an', 'in',
            'but', 'his', 'her', 'was', 'have', 'with', 'take', 'with', 'that', 'do', 'be',
            'for', 'if', 'it', 'are', 'on', 'this', 'will', 'at', 'about', 'as', 'so', 'guy',
            'him', 'your', 'had', 'can', 'hes', 'from', 'me', 'its', 'shes', 'get', 'my'])

cv = CountVectorizer(stop_words = stop_str)
tfidf_transformer = TfidfTransformer()


#bag of words
X_bow = cv.fit_transform(X.values.astype('U'))
X_bow.shape

(20000, 40147)

In [17]:
#analysis of dataset
#class distribution, text length distribution, total unique words, most common words
import matplotlib.pyplot as plt

#print('The accuracy for Naive Bayes classifier using TF-IDF is {:.5f} on training data'.format(accuracy_score(y_pred = clf.predict(X_train), y_true = y_train)))
#pos_ct = sum(data['student_star'] > 0)
pos_ct = sum(y > 0)
#neg_ct = sum(data['student_star'] < 0)
neg_ct = sum(y == 0)

print('There are {:.0f} total records.'.format( pos_ct + neg_ct ))
print('There are {:.0f} positive records.'.format( pos_ct ))
print('There are {:.0f} negative records.'.format( neg_ct ))
print('{:.2f}% of records in the dataset are positive.\n'.format( (pos_ct / (pos_ct + neg_ct)) * 100 ))

'''
word_ct = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["word_comment"])
plt.hist(word_ct, color='lightgreen', ec='black', bins=10)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.title("Distribution of Word Count")
plt.show()
'''

print('\nThere are {:.0f} unique words.\n'.format( len(cv.get_feature_names_out()) ))

print('The 25 most common words, excluding stop words, are: ')
freqs = zip(cv.get_feature_names_out(), X_bow.sum(axis=0).tolist()[0])
# sort from largest to smallest
print( sorted(freqs, key=lambda x: -x[1]) [:25])

There are 20000 total records.
There are 16822 positive records.
There are 3178 negative records.
84.11% of records in the dataset are positive.


There are 40147 unique words.

The 25 most common words, excluding stop words, are: 
[('not', 10960), ('they', 9701), ('br', 9161), ('like', 8772), ('these', 8270), ('them', 7303), ('good', 6660), ('or', 6084), ('one', 6026), ('great', 5992), ('just', 5925), ('very', 5844), ('taste', 5639), ('all', 5592), ('coffee', 5427), ('product', 5098), ('we', 4952), ('when', 4788), ('flavor', 4773), ('tea', 4690), ('love', 4606), ('has', 4566), ('more', 4462), ('food', 4195), ('would', 4039)]


In [18]:
#training classifiers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#train-test split with Bag of Words, then apply TF-IDF to have same set of sample data
X_bow_train, X_bow_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2)

y_train_arr = y_train.to_numpy().reshape(-1)

X_tfidf_train = tfidf_transformer.fit_transform(X_bow_train)
X_tfidf_train.shape
X_tfidf_test = tfidf_transformer.fit_transform(X_bow_test)
X_tfidf_test.shape

print('Classifier\t Feature Extr.\t Train Acc.\t Test Acc.')
print('---------------------------------------------------------')

#naive bayes
from sklearn.naive_bayes import MultinomialNB
clf_bow = MultinomialNB().fit(X_bow_train, y_train_arr)
clf_tfidf = MultinomialNB().fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_bow.predict(X_bow_train), y_true = y_train),
                                                              accuracy_score(y_pred = clf_bow.predict(X_bow_test), y_true = y_test)))
print('Naive Bayes\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#support vector machine
from sklearn.linear_model import SGDClassifier
svm_bow = SGDClassifier().fit(X_bow_train, y_train_arr)
svm_tfidf = SGDClassifier().fit(X_tfidf_train, y_train_arr)
print('SVM\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_bow.predict(X_bow_train), y_true = y_train),
                                                          accuracy_score(y_pred = svm_bow.predict(X_bow_test), y_true = y_test)))
print('SVM\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))


print('---------------------------------------------------------')
#logistic regression
from sklearn.linear_model import LogisticRegression
lr_bow = LogisticRegression().fit(X_bow_train, y_train_arr)
lr_tfidf = LogisticRegression().fit(X_tfidf_train, y_train_arr)
print('Log Regression\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_bow.predict(X_bow_test), y_true = y_test)))
print('Log Regression\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_bow.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#linear support vector classification
from sklearn.svm import LinearSVC
svc_bow = LinearSVC(max_iter = 2000).fit(X_bow_train, y_train_arr)
svc_tfidf = LinearSVC().fit(X_tfidf_train, y_train_arr)
print('SVC\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = svc_bow.predict(X_bow_test), y_true = y_test)))
print('SVC\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))



Classifier	 Feature Extr.	 Train Acc.	 Test Acc.
---------------------------------------------------------
Naive Bayes	 BoW		 0.93506	 0.88775
Naive Bayes	 TF-IDF		 0.84075	 0.84300
---------------------------------------------------------
SVM		 BoW		 0.99550	 0.90450
SVM		 TF-IDF		 0.94812	 0.90950
---------------------------------------------------------
Log Regression	 BoW		 0.99138	 0.91275
Log Regression	 TF-IDF		 0.92263	 0.85100
---------------------------------------------------------
SVC		 BoW		 0.99975	 0.90100
SVC		 TF-IDF		 0.99056	 0.91500


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
#hyperparameter testing
from sklearn.model_selection import GridSearchCV
print('Testing for hyperparameters:')

#naive bayes
clf_param = {
    'fit_prior': (True, False),
    'force_alpha': (True, False),
    'alpha': (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1),
}

gs_clf = GridSearchCV(clf_tfidf, clf_param, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_tfidf_train, y_train_arr)
print('Naive Bayes:')
for param_name in sorted(clf_param.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print('---------------------------------------------------------')



#support vector machine
svm_param = {
    'max_iter': (1500, 1000, 500),
    'loss': ('hinge', 'perceptron', 'squared_error'),
    'alpha': (0.01, 0.005, 0.001),
}

gs_svm = GridSearchCV(svm_tfidf, svm_param, cv=5, n_jobs=-1)
gs_svm = gs_svm.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svm_param.keys()):
    print("%s: %r" % (param_name, gs_svm.best_params_[param_name]))
print('---------------------------------------------------------')



#logistic regression
lr_param = {
    'solver': ('lbfgs', 'liblinear', 'newton-cg'),
    'max_iter': (500, 750, 1000, 1250)
}

gs_lr = GridSearchCV(lr_tfidf, lr_param, cv=5, n_jobs=-1)
gs_lr = gs_lr.fit(X_tfidf_train, y_train_arr)
print('Logistic Regression:')
for param_name in sorted(lr_param.keys()):
    print("%s: %r" % (param_name, gs_lr.best_params_[param_name]))
print('---------------------------------------------------------')



#linear support vector classification
svc_param = {
    'max_iter': (500, 750, 1000, 2000, 2500),
    'C': (0.01, 0.1, 0.5, 1.0, 1.5),
}

gs_svc = GridSearchCV(svc_tfidf, svc_param, cv=5, n_jobs=-1)
gs_svc = gs_svc.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svc_param.keys()):
    print("%s: %r" % (param_name, gs_svc.best_params_[param_name]))
print('---------------------------------------------------------')

Testing for hyperparameters:
Naive Bayes:
alpha: 0.2
fit_prior: False
force_alpha: True
---------------------------------------------------------
SVM:
alpha: 0.005
loss: 'perceptron'
max_iter: 500
---------------------------------------------------------
Logistic Regression:
max_iter: 500
solver: 'lbfgs'
---------------------------------------------------------
SVM:
C: 1.0
max_iter: 500
---------------------------------------------------------


In [20]:
#testing the optimized hyperparameters with TF-IDF
print('Results of applying tuned hyperparameters')
print('-----------------------------------------')

print('Classifier\tTrain Acc.\tTest Acc.')
clf_tfidf = MultinomialNB(alpha = 0.6).fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

svm_tfidf = SGDClassifier(alpha = 0.001, loss = 'squared_error', max_iter = 1500).fit(X_tfidf_train, y_train_arr)
print('SVM\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))

lr_tfidf = LogisticRegression(solver = 'liblinear', max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('Log Regression\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_bow.predict(X_tfidf_test), y_true = y_test)))

svc_tfidf = LinearSVC(C = 0.1, max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('SVC\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))

Results of applying tuned hyperparameters
-----------------------------------------
Classifier	Train Acc.	Test Acc.
Naive Bayes	 0.84481	 0.84275
SVM		 0.86769	 0.86425
Log Regression	 0.92306	 0.85100
SVC		 0.92375	 0.90000
